## 求导和梯度

In [ ]:
import numpy as np

In [ ]:
# 函数fx在x点上的导数，这里用中心差分
def numerical_diff(f, x):
    # 偏差不能太小 计算机由于精度原因会把太小的值当作0
    delta_x = 1e-4 
    # 由于delta_x不能无限小，因此用中心差分来减少误差
    # dfx/dx = (f(x + delta) - f (x - delta)) / 2h
    return (f(x + delta_x) - f(x - delta_x)) / (2 * delta_x)

f = lambda x: x ** 2 + 3 * x + 100
numerical_diff(f, 10)


标量y对向量求导,即为该向量的梯度， 应该需要转置，即行向量的梯度为列向量

但是书上没有转制，大概后续乘法会交换顺序把。

这里按照书上来，先不转置

In [ ]:
# x是向量
def numerical_gradient(f, x):
    delta_x = 1e-4
    grad = np.zeros_like(x)
    
    # 遍历x，分别对每一个参数求导，
    # 实际上就是对当前的x[i]进行中心分差计算，其他元素不变
    for i in range(x.size):
        temp_x_i = x[i]
        x[i] = temp_x_i + delta_x
        fx1 = f(x)
        x[i] = temp_x_i - delta_x
        fx2 = f(x)
        grad[i] = (fx1 - fx2) / (2 * delta_x)
        x[i] = temp_x_i
    
    return grad

# 测试梯度函数
f = lambda a: a[0] ** 2 + 2 *a[0] * a[1] + a[2] ** 2
numerical_gradient(f, np.array([3.0, 4.0, 5.0]))


接下来可以写出梯度下降法的函数了:x = x - lr * grad_x  
x是向量，其中lr是学习率，grad_x是x的梯度  
分解开就是:
    x[0] = x[0] - lr * grad_x[0]    
    x[1] = x[1] - lr * grad_x[1]
    ... 
    x[n] = x[n] - lr * grad_x[n]  
重复一定次数即可，求出fx的极小值/最小值了

In [ ]:
# 梯度下降求函数的fx极小值的x坐标，lr系数，学习率 step重复次数
def gradient_descent(f, init_x, lr = 0.01, step=100):
    x = init_x
    for i in range(step):
        grad = numerical_gradient(f, x)
        x = x - lr * grad
    return x

# 测试下梯度下降
fx = lambda x: x[0] ** 2 + x[1] ** 2
init_x = np.array([33.0, 20.0])
gradient_descent(fx, init_x, lr=0.1)

定义一个简单的单层网络类，权重w是一个2 * 3的矩阵，偏置先不设置，激活函数是softmax

可以计算出这个网络的损失函数

In [ ]:
# 激活函数
def softmax(x):
    c = np.max(x)
    exp_a = np.exp(x - c) # 减去c是为了防止溢出
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

# 损失函数 交叉熵误差
def cross_entropy_error(y, t):
    # 由于loge(0)是负无穷大-inf，计算机无法继续之后的运算
    # 所以给输入增加一个微小的数，并且不影响结果
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

class SimpleNet:
    
    # 初始化网络    
    def __init__(self):
        super().__init__()
        self.w = np.random.randn(2, 3)

    # 推理一次 即forward    
    def predict(self, x):
        y = np.dot(x, self.w)
        y = softmax(y)
        return y

    # 网络的损失
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
        

In [ ]:
# 设置np的打印选项：精度最大为小数点后4位，不用科学计数法
np.set_printoptions(precision=4, suppress=True)

# 简单测试下网络
net = SimpleNet()
x = np.array([[0.6, 0.9]])
t = np.array([0.0, 0.0, 1.0])
d = net.loss(x, t)
print('loss:', d)


# 一个通用的算术求梯度，遍历ndarray进行运算
def numerical_gradient_common(f, x):
    h = 1e-4 
    grad = np.zeros_like(x)
    
    # 利用迭代器遍历ndarray，这样才能保证拿到索引并且有修改ndarray的权限
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    for _ in it:  # 也可以这么写while not it.finished:
        idx = it.multi_index
        temp = x[idx]
        x[idx] = temp + h
        fx1 = f(x)
        x[idx] = temp - h
        fx2 = f(x)
        grad[idx] = (fx1 - fx2) / (2 * h)
        x[idx] = temp
    return grad

# loss函数输入x t，再通过与w b的计算 可以表示为y = f(w) + f(b)
# 因此这里损失对w求导，直接用loss_w即可，求导函数里每次调用f(x)的时候，会按照当前w b x t计算一次
# 而我们传入的net.w，进行了微小的更改+h和-h，这里就是损失对w求偏导了
# 可以测试一下
loss_w = lambda w: net.loss(x, t)
grad = numerical_gradient_common(loss_w, net.w)

print('grad:', grad)

# w[0][0]增加1 看对应的loss增加多少
net.w[0][0] += 1
d = net.loss(x, t)
print('loss:', d)
# loss应该是差不多增加了grad[0][0] * w[0][0],

接下来测试下sgd随机梯度下降

In [ ]:
# 给一组真实数据
true_x = np.array([[0.2, 0.3]])
true_w = np.array([[0.3, -0.5, 1.2], [0.2, 1.2, 0.5]])
true_y = true_x.dot(true_w)
true_y = softmax(true_y)
max = np.argmax(true_y)
true_y = np.zeros_like(true_y)
true_y[0][max] = 1

nn = SimpleNet()
d = nn.loss(true_x, true_y)
print('loss 1:', d)
loss_nn = lambda _: nn.loss(true_x, true_y)

# sgd 
lr = 0.1
step = 1000
for _ in range(step):
    grad = numerical_gradient_common(loss_nn, nn.w)
    nn.w -= lr * grad
    
    
# 打印sgd之后的误差 和推理结果
d = nn.loss(true_x, true_y)
print('loss 2:', d)
y_hat = nn.predict(true_x)
print('y_hat:', y_hat)

